## TORCH.JIT.TRACE_MODULE

<b>
    
```python
torch.jit.trace_module(mod, inputs, optimize=None, check_trace=True, check_inputs=None, check_tolerance=1e-05, strict=True, _force_outplace=False, _module_class=None, _compilation_unit=<torch.jit.CompilationUnit object>)
```
</b>
    

### Parameters
1. mod :- A ```torch.nn.Module``` containing methods whose names are specified in ```inputs```. The methods will be compiled as a part of a single ```ScriptModule```.
2. inputs :- A dict containing sample inputs indexed by method names in ```mod```. The inputs will be passed to methods whose names correspond to inputs’ keys while tracing.
```python
{'forward' : forward_input, 'method2': method_input}
```

### Returns

A ```ScriptModule``` with a single ```forward``` method containing the traced code. When ```func``` is a ```torch.nn.Module```, the returned ```ScriptModule``` will have the same set of sub-modules and parameters as ```func```.


### Example (Tracing a Module with Multiple Methods)

In [3]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv = nn.Conv2d(1, 1, 3)
        
    def forward(self, x):
        return self.conv(x)
    
    def weighted_kernel_sum(self, weight):
        return weight * self.conv.weight
    
model = Net()
weight = torch.rand(1, 1, 3, 3)
forward_input = torch.rand(1, 1, 3, 3)

# Trace a specific method and construct `ScriptModule` with
# a single `forward` method
module = torch.jit.trace(model.forward, forward_input)
print('Specific Method Traced Code: ', module.code)
print('Specific Method Graph: ', module.graph)

# Trace a module (implicitly traces `forward`) and construct a
# `ScriptModule` with a single `forward` method
module = torch.jit.trace(model, forward_input)
print('Traced Code: ', module.code)
print('Implicit Method Graph: ', module.graph)

# Trace specific methods on a module (specified in `inputs`), constructs
# a `ScriptModule` with `forward` and `weighted_kernel_sum` methods
inputs = {'forward' : forward_input, 'weighted_kernel_sum' : weight}
module = torch.jit.trace_module(model, inputs)
print('Specific Method Inputs Traced Code: ', module.code)
print('Specific Method Inputs Graph: ', module.graph)

Specific Method Traced Code:  def forward(self,
    x: Tensor) -> Tensor:
  return (self.conv).forward(x, )

Specific Method Graph:  graph(%self.1 : __torch__.___torch_mangle_27.Net,
      %x : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu)):
  %31 : __torch__.torch.nn.modules.conv.___torch_mangle_26.Conv2d = prim::GetAttr[name="conv"](%self.1)
  %33 : Tensor = prim::CallMethod[name="forward"](%31, %x)
  return (%33)

Traced Code:  def forward(self,
    x: Tensor) -> Tensor:
  return (self.conv).forward(x, )

Implicit Method Graph:  graph(%self.1 : __torch__.___torch_mangle_31.Net,
      %x : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu)):
  %31 : __torch__.torch.nn.modules.conv.___torch_mangle_30.Conv2d = prim::GetAttr[name="conv"](%self.1)
  %33 : Tensor = prim::CallMethod[name="forward"](%31, %x)
  return (%33)

Specific Method Inputs Traced Code:  def forward(self,
    x: Tensor) -> Tensor:
  return (self.conv).forward(x, )

Specific Met